# 🎬 Sailing Agent Trajectory Visualizer

This notebook provides an interactive interface for visualizing sailing agent trajectories.

**Features:**
1. **Single Agent Visualization**: Watch your agent navigate through the windfield step-by-step
2. **Multi-Agent Race Mode**: Compare multiple agents racing on the same windfield! 🏁


In [1]:
# If you've modified visualization.py, run this cell to reload it
import importlib
import sys
if 'wind_scenarios.visualization' in sys.modules:
    import wind_scenarios.visualization
    importlib.reload(wind_scenarios.visualization)
    print("✅ Reloaded visualization module")


## Setup


In [3]:
import sys
import os
import numpy as np
import pandas as pd
from IPython.display import display

# # Add the src directory to the path
# sys.path.append(os.path.abspath('../src'))
# sys.path.append(os.path.abspath('..'))

# Import the evaluation and visualization tools
from wind_scenarios.test_agent_validity import validate_agent
from wind_scenarios.evaluation import evaluate_agent, visualize_trajectory
from wind_scenarios.visualization import visualize_race, print_race_summary, create_race_gif
from wind_scenarios import get_wind_scenario, WIND_SCENARIOS

# List available wind scenarios
print("Available windfields:")
for windfield_name in sorted(WIND_SCENARIOS.keys()):
    print(f"  - {windfield_name}")

print("\n✅ Setup complete!")


Available windfields:
  - simple_static
  - static_headwind
  - training_1
  - training_2
  - training_3

✅ Setup complete!


---
# Part 1: Single Agent Trajectory Viewer

Watch a single agent navigate through the windfield with an interactive slider.


## Configuration


In [4]:
#############################################
### CONFIGURE YOUR VISUALIZATION HERE ######
#############################################

# Agent to visualize
AGENT_PATH = "../src/submission/my_agent_dqn_120k.py"

# Windfield to use
WINDFIELD_NAME = "simple_static"

# Seed for reproducibility
SEED = 1

# Maximum steps
MAX_STEPS = 200

#############################################

print(f"Agent: {AGENT_PATH}")
print(f"Windfield: {WINDFIELD_NAME}")
print(f"Seed: {SEED}")
print(f"Max steps: {MAX_STEPS}")
validation_results = validate_agent(AGENT_PATH)
validation_results


Agent: ../src/submission/my_agent_dqn_120k.py
Windfield: simple_static
Seed: 1
Max steps: 200


{'valid': True,
 'agent_class': my_agent_dqn_120k.MyAgent,
 'agent_name': 'MyAgent',
 'errors': [],
 'warnings': ['Agent does not implement save() method',
  'Agent does not implement load() method']}

## Load and Run Agent


In [5]:
# Load agent
validation_results = validate_agent(AGENT_PATH)
if not validation_results['valid']:
    print("❌ Agent validation failed:")
    for error in validation_results['errors']:
        print(f"  - {error}")
    raise ValueError("Invalid agent")

AgentClass = validation_results['agent_class']
agent = AgentClass()
print(f"✅ Loaded agent: {AgentClass.__name__}")

# Get windfield configuration
wind_scenario = get_wind_scenario(WINDFIELD_NAME)
wind_scenario['env_params'] = {
    'wind_grid_density': 32,  # Show all 32x32 arrows
    'wind_arrow_scale': 120,  # Higher scale = shorter arrows (prevents overlap)
    'render_mode': "rgb_array",
    'show_full_trajectory': True # show full trajectory 
}

# Run evaluation with rendering
print(f"\n🎬 Running agent on {WINDFIELD_NAME} with seed {SEED}...")
results = evaluate_agent(
    agent=agent,
    wind_scenario=wind_scenario,
    seeds=SEED,
    max_horizon=MAX_STEPS,
    verbose=False,
    render=True,
    full_trajectory=True
)

# Display results
print(f"\n📊 Results:")
print(f"  Reward: {results['rewards'][0]:.2f}")
print(f"  Steps: {results['steps'][0]}")
print(f"  Success: {'✅' if results['individual_results'][0]['success'] else '❌'}")
print(f"  Frames captured: {len(results['frames'])}")


✅ Loaded agent: MyAgent

🎬 Running agent on simple_static with seed 1...

📊 Results:
  Reward: 100.00
  Steps: 125
  Success: ✅
  Frames captured: 125


In [6]:
# Choose which wind scenarios to evaluate on
TRAINING_WIND_SCENARIOS = ["simple_static", "static_headwind", "training_1", "training_2", "training_3"]

# Evaluation parameters for all wind scenarios
ALL_SEEDS = [42, 43, 44, 45, 46]  # Seeds to use for all evaluations
ALL_MAX_HORIZON = 200             # Maximum steps per episode

#############################################
### DO NOT MODIFY BELOW THIS LINE ##########
#############################################


all_results = {}
    
print(f"Evaluating agent on {len(TRAINING_WIND_SCENARIOS)} wind scenarios (including simple static)...")
    
    # Evaluate on each wind scenario
for wind_scenario_name in TRAINING_WIND_SCENARIOS:
    print(f"\nWind scenario: {wind_scenario_name}")
        
        # Get the wind scenario
    wind_scenario = get_wind_scenario(wind_scenario_name)
        
        # Run the evaluation
    results = evaluate_agent(
            agent=agent,
            wind_scenario=wind_scenario,
            seeds=ALL_SEEDS,
            max_horizon=ALL_MAX_HORIZON,
            verbose=False,  # Less verbose for multiple evaluations
            render=True,
            full_trajectory=True
        )
        
        # Store results
    all_results[wind_scenario_name] = results
        
        # Print summary
    print(f"  Success Rate: {results['success_rate']:.2%}")
    print(f"  Mean Reward: {results['mean_reward']:.2f}")
    print(f"  Mean Steps: {results['mean_steps']:.1f}")
    
    # Print overall performance
total_success = sum(r['success_rate'] for r in all_results.values()) / len(all_results)
print("\n" + "="*50)
print(f"OVERALL SUCCESS RATE: {total_success:.2%}")
print("="*50)

Evaluating agent on 5 wind scenarios (including simple static)...

Wind scenario: simple_static
  Success Rate: 20.00%
  Mean Reward: 6.05
  Mean Steps: 184.0

Wind scenario: static_headwind
  Success Rate: 80.00%
  Mean Reward: 36.73
  Mean Steps: 103.0

Wind scenario: training_1
  Success Rate: 100.00%
  Mean Reward: 55.83
  Mean Steps: 60.2

Wind scenario: training_2
  Success Rate: 100.00%
  Mean Reward: 76.39
  Mean Steps: 27.8

Wind scenario: training_3
  Success Rate: 100.00%
  Mean Reward: 59.90
  Mean Steps: 52.6

OVERALL SUCCESS RATE: 80.00%


## Interactive Trajectory Viewer


In [6]:
# Display interactive slider
visualize_trajectory(results, None, with_slider=True)


interactive(children=(IntSlider(value=0, description='Step:', max=124), Output()), _dom_classes=('widget-inter…

---
# Part 2: Multi-Agent Race Viewer 🏁

Watch multiple agents race against each other on the same windfield!

**Note:** All agents will face the exact same wind conditions (same windfield + seed).


## Configuration


In [7]:
#############################################
### CONFIGURE YOUR RACE HERE ###############
#############################################

# Agents to race (add as many as you want!)
RACE_AGENTS = [
    {"path": "../submissions/my_agent_fast.py", "name": "Agent Best", "color": "purple"},
    {"path": "../src/submission/my_agent_dqn_120k.py", "name": "Agent DQN 120k", "color": "orange"},
    {"path": "../submissions/my_agent_dqn_80k.py", "name": "Agent dqn 80k", "color": "black"},
    {"path": "../submissions/my_agent_noisy_noisynet_per_12k.py", "name": "Agent new dqn noisynet", "color": "yellow"},
    {"path": "../submissions/my_agent_new_train_dqn_baseline.py", "name": "Agent dqn 20K", "color": "red"},
    {"path": "../src/submission/my_agent_finale.py", "name": "Agent finale", "color": "blue"},
    {"path": "../submissions/my_agent_dqn_noisynet.py", "name": "Agent dqn noisynet", "color": "gray"}
]

# Race windfield
RACE_WINDFIELD = "static_headwind"

# Race seed
RACE_SEED = 1

# Maximum steps
RACE_MAX_STEPS = 200

#############################################

print(f"🏁 Race Configuration:")
print(f"  Windfield: {RACE_WINDFIELD}")
print(f"  Seed: {RACE_SEED}")
print(f"  Agents: {len(RACE_AGENTS)}")
for i, agent_info in enumerate(RACE_AGENTS):
    print(f"    {i+1}. {agent_info['name']} ({agent_info['color']})")


🏁 Race Configuration:
  Windfield: static_headwind
  Seed: 1
  Agents: 7
    1. Agent Best (purple)
    2. Agent DQN 120k (orange)
    3. Agent dqn 80k (black)
    4. Agent new dqn noisynet (yellow)
    5. Agent dqn 20K (red)
    6. Agent finale (blue)
    7. Agent dqn noisynet (gray)


## Run All Agents


In [8]:
# Load and run all agents
race_results = []

for agent_info in RACE_AGENTS:
    print(f"\n🤖 Loading {agent_info['name']}...")
    
    # Load agent
    validation_results = validate_agent(agent_info['path'])
    if not validation_results['valid']:
        print(f"  ❌ Validation failed for {agent_info['name']}")
        continue
    
    AgentClass = validation_results['agent_class']
    agent = AgentClass()
    
    # Get windfield (must be same for all agents)
    wind_scenario = get_wind_scenario(RACE_WINDFIELD)
    
    # Run evaluation (without rendering individual frames, we'll render the race custom)
    results = evaluate_agent(
        agent=agent,
        wind_scenario=wind_scenario,
        seeds=RACE_SEED,
        max_horizon=RACE_MAX_STEPS,
        verbose=False,
        render=False,  # We'll do custom rendering
        full_trajectory=True
    )
    
    # Store results with agent info
    race_results.append({
        'name': agent_info['name'],
        'color': agent_info['color'],
        'positions': results['positions'],
        'actions': results['actions'],
        'reward': results['rewards'][0],
        'steps': results['steps'][0],
        'success': results['individual_results'][0]['success']
    })
    
    print(f"  ✅ Reward: {results['rewards'][0]:.2f}, Steps: {results['steps'][0]}, Success: {results['individual_results'][0]['success']}")

print(f"\n✅ All {len(race_results)} agents completed!")



🤖 Loading Agent Best...
  ✅ Reward: 100.00, Steps: 81, Success: True

🤖 Loading Agent DQN 120k...
  ✅ Reward: 100.00, Steps: 79, Success: True

🤖 Loading Agent dqn 80k...
  ✅ Reward: 100.00, Steps: 71, Success: True

🤖 Loading Agent new dqn noisynet...
  ✅ Reward: 100.00, Steps: 55, Success: True

🤖 Loading Agent dqn 20K...
  ✅ Reward: 100.00, Steps: 74, Success: True

🤖 Loading Agent finale...
  ✅ Reward: 100.00, Steps: 52, Success: True

🤖 Loading Agent dqn noisynet...
  ✅ Reward: 0.00, Steps: 200, Success: False

✅ All 7 agents completed!


## Race Visualization

Use the slider to watch all agents move through the windfield simultaneously!


In [11]:
# Visualize the race using the visualization module!
# Visualize the race using the visualization module!
visualize_race(race_results, RACE_WINDFIELD, RACE_SEED, RACE_MAX_STEPS, 
               show_full_trajectories=True)  # ← Add this parameter

interactive(children=(IntSlider(value=0, description='Race Step:', max=50), Output()), _dom_classes=('widget-i…

## Race Summary


In [9]:
# Display race summary using the visualization module
print_race_summary(race_results)



🏆 RACE RESULTS 🏆


,Agent,Color,Steps,Reward,Success
5,Agent finale,blue,52,100.00,✅
3,Agent new dqn noisynet,yellow,55,100.00,✅
2,Agent dqn 80k,black,71,100.00,✅
4,Agent dqn 20K,red,74,100.00,✅
1,Agent DQN 120k,orange,79,100.00,✅
0,Agent Best,purple,81,100.00,✅
6,Agent dqn noisynet,gray,200,0.00,❌



🥇 WINNER: Agent finale (completed in 52 steps!)


## Create GIF Animation

Run this cell to export the race as an animated GIF file!


In [10]:
#############################################
### GIF CONFIGURATION #######################
#############################################

# Enable/disable GIF creation
CREATE_GIF = True  # Set to True to create a GIF

# GIF settings
GIF_OUTPUT_PATH = "race_animation.gif"  # Where to save the GIF
GIF_FPS = 15  # Frames per second (higher = faster animation)
GIF_STEP_INTERVAL = 1  # 1 = every step, 2 = every other step (reduces file size)

#############################################

if CREATE_GIF:
    if 'race_results' not in locals():
        print("⚠️ Please run the race evaluation cells first!")
    else:
        create_race_gif(
            race_results=race_results,
            windfield_name=RACE_WINDFIELD,
            seed=RACE_SEED,
            output_path=GIF_OUTPUT_PATH,
            fps=GIF_FPS,
            step_interval=GIF_STEP_INTERVAL,
            show_full_trajectories=True  # ← Add this parameter
        )
else:
    print("ℹ️ GIF creation is disabled. Set CREATE_GIF = True to enable.")


🎬 Creating race GIF...


/home/hamidou-diallo/Dev/Academic/sailling-challenge/wind_scenarios/visualization.py:462: UserWarning: Glyph 9989 (\N{WHITE HEAVY CHECK MARK}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/home/hamidou-diallo/Dev/Academic/sailling-challenge/wind_scenarios/visualization.py:466: UserWarning: Glyph 9989 (\N{WHITE HEAVY CHECK MARK}) missing from font(s) DejaVu Sans.
  plt.savefig(buf, format='png', dpi=100, bbox_inches='tight')


  Processed 200/200 frames...
💾 Saving GIF to race_animation.gif...
✅ GIF created successfully! Saved to: race_animation.gif
   Total frames: 200 | Duration: ~13.3 seconds


**GIF Parameters:**
- `GIF_FPS`: Controls animation speed (10 = normal, 15 = faster, 5 = slower)
- `GIF_STEP_INTERVAL`: Sample every N steps (1 = all frames, 2 = half frames)
- Higher step interval = smaller file size but less smooth

**Note:** You need to install `imageio` to create GIFs:
```bash
pip install imageio
```
